# Parkingson's Dataset (Machine Learning)
# Scikit-Learn (Random Forests Regressor & Gradient Boosting Regressor) Regression

Author: James Roberts
Date: 4/22/2022

The “Parkinson’s Dataset” is a collection of voice measurements from patients with early-stage
Parkinson’s disease (PD) who were recruited to a six-month trial of a tele-monitoring device for
remote symptom progression monitoring. The recordings were automatically captured in the patients’ homes. The dataset was created by Drs. Athanasios Tsanas and Max Little of the University of Oxford, UK, in collaboration with 10 medical centers in the USA and Intel Corporation, who developed the tele-monitoring device to record the speech signals. The Parkinson’s
Dataset was donated by the authors on June, 2008 and can be retrieved at the public URL:
https://archive.ics.uci.edu/ml/datasets/Parkinsons.

Includes 19 features, which are detailed below:
name: integer that uniquely identifies each subject participating to the study.
Jitter(%), Jitter(Abs), Jitter:RAP, Jitter:PPQ5, Jitter:DDP: A total of five (5) measures of the variation in fundamental frequency (in Hz) of the speech sound.
Shimmer, Shimmer(dB), Shimmer:APQ3, Shimmer:APQ5, Shimmer:APQ11, Shimmer:DDA: A
total of six (6) measures of variation in amplitude of the speech sound.
NHR, HNR: two (2) measures of ratio of noise-to-tonal components in the voice.
RPDE: a nonlinear dynamical complexity measure.
DFA: signal fractal scaling exponent.
PPE: a nonlinear measure of fundamental frequency variation.
motor UPDRS: UPDRS motor score, which is a numerical score assigned by the clinician upon
examining the motor abilities of the patient in a sequence of predefined motor tasks.
total UPDRS: UPDRS total score, which is the sum of multiple scores assigned by the clinician
upon examining the motor abilities, mental abilities, and mood of the patient in a sequence
of predefined tasks.
A description of the databases is reported in article [1]. A total of 5,875 voice recordings from 42
patients are included in the dataset, approximately 200 recordings per patient. No missing value
is reported.

[1] Tsanas A., Little M. A., McSharry P. E., Ramig L. O., “Accurate telemonitoring of Parkinson’s
disease progression by noninvasive speech tests,” IEEE Trans Biomed Eng, vol. 57 (4), pp. 884–
93, April 2010. DOI: 10.1109/TBME.2009.2036000

In [5]:
import pandas as pd             # Pandas is for data analysis and structure manipulation
import numpy as np              # NumPy is for numerical operations
import matplotlib               # MatPlotLib is for making plots & figures
import matplotlib.pyplot as plt # PyPlot is a subset of the library for making MATLAB-style plots

## Let us setup the font size
plt.rcParams['axes.labelsize']  = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
print('Done!')

import warnings
warnings.filterwarnings("ignore")

Done!


In [6]:
# Upload Files (if using google colab, otherwise comment out)
from google.colab import files
import io
uploaded = files.upload()

In [7]:
# Create data table
dataTable = pd.read_csv('dataset_parkinson.csv')

In [8]:
# Retrieve the list of patients
P = dataTable['name'].to_numpy()
P = P.astype('int')

# Retrieve the output variable and the feature matrix
# Output Variable: Y
Y = dataTable['motor_UPDRS'].to_numpy()

# Feature Matrix without the output variable. Remove names. Remove total_UPDRS
# as it is based on our Output variable
dataTable = dataTable.drop(['motor_UPDRS','total_UPDRS','name'], axis = 1)
X = dataTable.to_numpy()

print('Size of Y: %d (min: %d - max: %d)' % (np.size(Y,axis=0),np.min(Y),np.max(Y)))
print('Size of X: %d x %d' % (np.size(X,axis=0),np.size(X,axis=1)))

Size of Y: 5875 (min: 5 - max: 39)
Size of X: 5875 x 16


# Design an Optimal Regressor

In [9]:
# 70-30 split and use the training set to optimize the regressors. The splitting is done on the patient ID
# after shuffling the sequence of patients IDs
patientID = np.arange(42)
np.random.shuffle(patientID)
ncut = (np.floor(42*0.7)).astype('int')

# IDs used for training and testing
ptn_train = patientID[:ncut]
ptn_test = patientID[ncut:]

# Build the training set
n_features = np.size(X,axis=1)
X_train = np.empty([0,n_features],dtype='float64')
Y_train = np.empty([0,1],dtype='float64')
for i in ptn_train:
    tmp = (P==i)
    X_train = np.vstack((X_train,X[tmp,:]))
    Y_train = np.vstack((Y_train,Y[tmp].reshape(-1,1)))

from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)
Y_train = StandardScaler().fit_transform(Y_train)

print('Size of Y_train: %d (min: %d - max: %d)' % (np.size(Y_train,axis=0),np.min(Y_train),np.max(Y_train)))
print('Size of X_train: %d x %d' % (np.size(X_train,axis=0),np.size(X_train,axis=1)))

# Build the test set
X_test = np.empty([0,n_features],dtype='float64')
Y_test = np.empty([0,1],dtype='float64')
for i in ptn_test:
    tmp = (P==i)
    X_test = np.vstack((X_test,X[tmp,:]))
    Y_test = np.vstack((Y_test,Y[tmp].reshape(-1,1)))
X_test = StandardScaler().fit_transform(X_test)
Y_test = StandardScaler().fit_transform(Y_test)

print('Size of Y_test: %d (min: %d - max: %d)' % (np.size(Y_test,axis=0),np.min(Y_test),np.max(Y_test)))
print('Size of X_test: %d x %d' % (np.size(X_test,axis=0),np.size(X_test,axis=1)))

Size of Y_train: 3923 (min: -1 - max: 2)
Size of X_train: 3923 x 16
Size of Y_test: 1802 (min: -1 - max: 1)
Size of X_test: 1802 x 16


In [13]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Search for the optimal Random Forests Regressor
obj1 = RandomForestRegressor(n_jobs=-1, random_state=36)



param_grid = {
    'criterion':['mae', 'mse'],
    'n_estimators': np.arange(30, 5000, 100),
    'min_samples_leaf': np.arange(1, 50, 5),
    'max_features':['auto', 'sqrt', 'log2']}

# set the number of folds for the cross-validation scheme used in the grid-search
n_folds = 3

searchRF = RandomizedSearchCV(obj1, param_grid, cv = n_folds, 
                              n_iter=5, scoring=['neg_mean_squared_error','r2','explained_variance'], 
                              refit='r2',
                              return_train_score=False, 
                              verbose=1)
searchRF.fit(X_train, Y_train)

# Save the parameters associated with "best R2 score" (i.e., the value specified in "refit") for future use
RF = searchRF.best_params_

# Create the optimal regressor
reg_rf = RandomForestRegressor(
    criterion= RF['criterion'], 
    n_estimators= RF['n_estimators'], 
    min_samples_leaf= RF['min_samples_leaf'],
    max_features= RF['max_features'], 
    n_jobs=-1, 
    random_state=36)

# Check the performance on the test data
reg_rf.fit(X_train,Y_train)
Y_RF = reg_rf.predict(X_test)
rmse_RF = np.sqrt(mean_squared_error(Y_test,Y_RF))
perc_RF = rmse_RF / (np.max(Y_test) - np.min(Y_test))
print('RMSE Error: %2.2f (abs. val) - %2.2f %%' % (rmse_RF,100*perc_RF))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
RMSE Error: 1.01 (abs. val) - 27.46 %


In [15]:
# Search for the optimal Gradient Boosting Regressor
obj2 = GradientBoostingRegressor(random_state=36)

param_grid = {
    'loss': ['ls', 'lad', 'huber', 'quantile'],
    'learning_rate': np.arange(0.1, 1.2, 0.05),
    'n_estimators': np.arange(20, 5000, 100),
    'subsample': np.arange(0.5, 1, 0.1),
    'criterion': ['friedman_mse', 'mse', 'mae'],
    'min_samples_leaf': np.arange(1, 50, 5),
    'max_depth': np.arange(3, 30, 3),
    'max_features':['auto', 'sqrt', 'log2']}

searchGB = RandomizedSearchCV(obj2, param_grid, cv = n_folds, 
                              n_iter=5, scoring=['neg_mean_squared_error','r2','explained_variance'], 
                              refit='neg_mean_squared_error',
                              return_train_score=False, 
                              verbose=1)
searchGB.fit(X_train, Y_train)

# Save the parameters associated with "best mean-squared error" (i.e., the value specified in "refit") for future use
GB = searchGB.best_params_

# Create the optimal regressor
reg_gb = GradientBoostingRegressor(
    loss=GB['loss'],
    learning_rate=GB['learning_rate'],
    n_estimators=GB['n_estimators'],
    subsample=GB['subsample'],
    criterion=GB['criterion'],
    min_samples_leaf=GB['min_samples_leaf'],
    max_depth=GB['max_depth'],
    max_features=GB['max_features'], 
    random_state=36)

# Check the performance on the test data
reg_gb.fit(X_train,Y_train)
Y_GB = reg_gb.predict(X_test)
rmse_GB = np.sqrt(mean_squared_error(Y_test,Y_GB))
perc_GB = rmse_GB / (np.max(Y_test) - np.min(Y_test))
print('RMSE Error: %2.2f (abs. val) - %2.2f %%' % (rmse_GB,100*perc_GB))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
RMSE Error: 1.12 (abs. val) - 30.41 %


# Test the Optimal Designs


In [18]:
# train and test the two regressors
reg_rf.fit(X_train,Y_train)
reg_gb.fit(X_train,Y_train)

Y_RF = reg_rf.predict(X_test)
Y_GB = reg_gb.predict(X_test)

# assess the performance
# Root Mean Squared Error
rmse_RF = np.sqrt(mean_squared_error(Y_test,Y_RF))
# Normalized Root Mean Squared Error - Value of 0 represents a perfect fiting model
perc_RF = rmse_RF / (np.max(Y_test) - np.min(Y_test))
print('RF: RMSE Error is %2.2f (abs. val) - %2.2f %%' % (rmse_RF,100*perc_RF))

# Root Mean Squared Error
rmse_GB = np.sqrt(mean_squared_error(Y_test,Y_GB))
# Normalized Root Mean Squared Error
perc_GB = rmse_GB / (np.max(Y_test) - np.min(Y_test))
print('GB: RMSE Error is %2.2f (abs. val) - %2.2f %%' % (rmse_GB,100*perc_GB))

RF: RMSE Error is 1.01 (abs. val) - 27.46 %
GB: RMSE Error is 1.12 (abs. val) - 30.41 %


From here, the most promising regressor is the Random Forests since it outperforms the GradientBoosting regressor. The RMSE could possibly be reduced by removing features that do not correlate well with the output.